In [1]:
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions

### MobileNet
- Neural network which is around 28MB. 
- Runs on mobile devices.
- Single-shot detection.

In [2]:
 model = MobileNet() # weights="path to the h5 file"

### Bring in an image to this model (do inference on a trained model)

In [3]:
import matplotlib.pyplot as plt

In [4]:
from tensorflow.keras.preprocessing import image

In [5]:
img = image.load_img('./data/hotdog/train/hot_dog/1017226.jpg', target_size=(224,224))
img_array = image.img_to_array(img) # Converting from PIL object into a numpy array
img_array.shape

(224, 224, 3)

In [6]:
probas = model.predict(img_array.reshape(1, 224,224,3))

In [7]:
decode_predictions(probas)

[[('n02782093', 'balloon', 0.4686488),
  ('n09256479', 'coral_reef', 0.10136139),
  ('n02606052', 'rock_beauty', 0.08158921),
  ('n04209239', 'shower_curtain', 0.0690787),
  ('n03530642', 'honeycomb', 0.031012459)]]

#### Load as a json file (if no access to internet)

In [8]:
#import json
#with open("json file path") as f:
#    imagenet = json.load(f)
# probas

## Transfer learning

- Bring back `MobileNet` and do not include the top layer on the network (the last layer excluded).

In [9]:
base_model = MobileNet()

#### Take the output of the last but one layer, and use it to build on top of it.

In [10]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, Sequential

In [11]:
len(base_model.layers)

93

In [12]:
new_model = Sequential([base_model.layers[i] for i in range(92)])
new_model.add(Dense(512, activation='relu'))
new_model.add(Dense(2, activation='softmax'))

In [ ]:
#new_new_model = Sequential(base_model.layers[::-1]+[Dense(512, activation='relu'), Dense(2, activation='softmax')])

#### Use generator to bring in the training images for hotdog/not hotdog 

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [23]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input
                                   , horizontal_flip=True, vertical_flip=True)

In [24]:
train_generator = train_datagen.flow_from_directory('./data/hotdog/train/', 
                                                    target_size=(224,224), 
                                                    color_mode='rgb', 
                                                    class_mode='categorical', 
                                                    batch_size=32)

Found 498 images belonging to 2 classes.


In [17]:
new_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
history = new_model.fit_generator(generator=train_generator,  epochs=10)

Epoch 1/10
16/16 [==============================] - 181s 11s/step - loss: 3.6764 - accuracy: 0.7349
Epoch 2/10
16/16 [==============================] - 197s 12s/step - loss: 2.1024 - accuracy: 0.6908
Epoch 3/10
16/16 [==============================] - 184s 12s/step - loss: 0.4238 - accuracy: 0.8353
Epoch 4/10
16/16 [==============================] - 182s 11s/step - loss: 0.4570 - accuracy: 0.8494
Epoch 5/10
16/16 [==============================] - 202s 13s/step - loss: 0.3638 - accuracy: 0.8775
Epoch 6/10
16/16 [==============================] - 211s 13s/step - loss: 0.4079 - accuracy: 0.8554
Epoch 7/10
16/16 [==============================] - 214s 13s/step - loss: 0.2807 - accuracy: 0.8936
Epoch 8/10
16/16 [==============================] - 209s 13s/step - loss: 0.2187 - accuracy: 0.9016
Epoch 9/10
16/16 [==============================] - 247s 15s/step - loss: 0.2871 - accuracy: 0.8956
Epoch 10/10
16/16 [==============================] - 206s 13s/step - loss: 0.1987 - accuracy: 0.9237

In [26]:
def load_img(file):
    img = image.load_img(file, target_size=(224,224))
    img_array = image.img_to_array(img) # Converting from PIL object into a numpy array
    return img_array.reshape(1, 224,224,3)

In [27]:
new_model.predict(load_img('./data/hotdog/test/hot_dog/138937.jpg'))

array([[6.053816e-09, 1.000000e+00]], dtype=float32)

In [ ]:
new